In [1]:
import os
import re
import time
import pandas as pd

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting


def generate(prompt):
    vertexai.init(project="lv-playground-genai", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-002",
    )
    responses = model.generate_content(
        [prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses


generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

In [ ]:
generate()
dataFinal=[]
folder = "2002/summaries/extracts_abstracts 2"
for j in os.listdir(folder)[67:]:
    if j != ".DS_Store" and "100" in os.listdir(f"{folder}/{j}"):
        with open(f"{folder}/{j}/100") as f:
            summary = [i.strip() for i in re.findall("<SUM.*?>(.*?)</SUM>",f.read().replace("\n"," "))][0]

        dataSection = []
        with open(f"{folder}/{j}/perdocs") as f:
            full_text = [i.strip() for i in re.findall("<SUM.*?>(.*?)</SUM>",f.read().replace("\n"," "))]

            for ft in full_text:
                row = {
                    "title": generate(f"What is the tittle for this text: {ft}, the expected output is just the title").text,
                    "Tanggal":"01/01/2002 00:00",
                    "Sumber Berita":f"{j}",
                    "content":ft,
                    "ringkasan": summary
                }
                dataSection.append(row)
                time.sleep(10)

            dataSection = pd.DataFrame(dataSection)

        full_docs = "\n".join(full_text)
        full_prompt = "What is the topix from these docs (Create in One Sentence): " + full_docs + "\n The expected output is the topic only"
        Topic = generate(full_prompt).text
        dataSection["Topik"] = Topic
        dataFinal.append(dataSection)
        pd.concat(dataFinal).to_csv("/content/drive/MyDrive/Tuna Swamp Optimizer/english_news.csv")
    time.sleep(10)
    print(f"Done {j}")